# Copyright Netherlands eScience Center <br>
** Function     : Packing netCDF for monthly sea ice concentration fields from NOAA/NSIDC (passive microwave)** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2019.10.03 ** <br>
** Last Update  : 2019.10.03 ** <br>
Description     : This notebook aims to pack the sea ice concentration fields from NOAA/NSIDC.<br>
Return Values   : netCDF4 <br>
Caveat          : The data is further processed by certain algorithms from NASA Goddard Center. More information is available through:<br>
https://nsidc.org/data/g02202<br>

In [1]:
import numpy as np
from netCDF4 import Dataset
import os
import glob

Initialization - Start with location of input and extraction of variables
Time span of each product:
- **SIC passive microwave** 1979 - 2018

In [2]:
################################   Input zone  ######################################
# specify starting and ending time
start_year = 1979
end_year = 2018
# specify data path
# SICpm fields
datapath = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/SIC_passive_microwave_NASA/monthly'
# sample
benchmark_key = Dataset(os.path.join(datapath,"seaice_conc_monthly_nh_n07_198604_v03r01.nc"))
# specify output path for figures
output_path = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/SIC_passive_microwave_NASA'
####################################################################################

In [5]:
latitude = benchmark_key.variables['ygrid'][:]
longitude = benchmark_key.variables['xgrid'][:]
print(latitude.shape)
print(longitude.shape)

(448,)
(304,)


In [3]:
def var_key_retrieve(datapath, year, month):
    # get the path to each datasets
    print ("Start retrieving datasets {} (y) - {} (m)".format(year,namelist_month[month-1]))
    # The shape of each variable is (304,448)
    # Sea Ice Concentration
    datapath_key = glob.glob(os.path.join(datapath,"seaice_conc_monthly_nh_*_{0}{1}_v03r01.nc".format(year,namelist_month[month-1])))[0]
    # get the variable keys
    var_key = Dataset(datapath_key)

    print ("Retrieving datasets successfully and return the variable key!")
    return var_key

In [4]:
def create_netcdf_point(pool_sicpm, output_path):
    print ('*******************************************************************')
    print ('*********************** create netcdf file*************************')
    print ('*******************************************************************')
    # wrap the datasets into netcdf file
    # 'NETCDF3_CLASSIC', 'NETCDF3_64BIT', 'NETCDF4_CLASSIC', and 'NETCDF4'
    data_wrap = Dataset(os.path.join(output_path, 'noaa_nsidc_monthly_regress_1979_2018_sic_passive_microwave.nc'),'w',format = 'NETCDF4')
    # create dimensions for netcdf data
    year_wrap_dim = data_wrap.createDimension('year',Dim_year)
    month_wrap_dim = data_wrap.createDimension('month',Dim_month)
    lat_wrap_dim = data_wrap.createDimension('latitude',Dim_latitude)
    lon_wrap_dim = data_wrap.createDimension('longitude',Dim_longitude)
    # create coordinate variables for 3-dimensions
    year_wrap_var = data_wrap.createVariable('year',np.int32,('year',))
    month_wrap_var = data_wrap.createVariable('month',np.int32,('month',))
    lat_wrap_var = data_wrap.createVariable('latitude',np.float32,('latitude',))
    lon_wrap_var = data_wrap.createVariable('longitude',np.float32,('longitude',))
    # create the actual 3-d variable
    sicpm_wrap_var = data_wrap.createVariable('sicpm',np.float32,('year','month','latitude','longitude'),zlib=True)
    # global attributes
    data_wrap.description = 'Monthly mean sea ice concentration with passive microwave by NOAA/NSIDC'
    # variable attributes
    lat_wrap_var.units = 'degree_north'
    lon_wrap_var.units = 'degree_east'

    sicpm_wrap_var.units = '1'

    sicpm_wrap_var.long_name = 'sea ice concentration with passive microwave'

    # writing data
    lat_wrap_var[:] = latitude
    lon_wrap_var[:] = longitude
    month_wrap_var[:] = index_month
    year_wrap_var[:] = period

    sicpm_wrap_var[:] = pool_sicpm

    # close the file
    data_wrap.close()
    print ("Create netcdf file successfully")

In [6]:
if __name__=="__main__":
    ####################################################################
    ######  Create time namelist matrix for variable extraction  #######
    ####################################################################
    # date and time arrangement
    # namelist of month and days for file manipulation
    namelist_month = ['01','02','03','04','05','06','07','08','09','10','11','12']
    # index of months
    period = np.arange(start_year,end_year+1,1)
    index_month = np.arange(1,13,1)
    latitude = benchmark_key.variables['ygrid'][:]
    longitude = benchmark_key.variables['xgrid'][:]
    ####################################################################
    ######       Extract invariant and calculate constants       #######
    ####################################################################
    # get invariant from benchmark file
    Dim_year = len(period)
    Dim_month = len(index_month)
    Dim_latitude = len(latitude)
    Dim_longitude = len(longitude)
    #############################################
    #####   Create space for stroing data   #####
    #############################################
    # data pool for zonal integral
    pool_sicpm = np.zeros((Dim_year,Dim_month,Dim_latitude,Dim_longitude),dtype = float)
    # loop for calculation
    for i in period:
        for j in index_month:
            # get the key of each variable
            var_key = var_key_retrieve(datapath, i, j)
            pool_sicpm[i-1979,j-1,:,:] = var_key.variables['goddard_merged_seaice_conc_monthly'][0,:,:]
    ####################################################################
    ######                 Data Wrapping (NetCDF)                #######
    ####################################################################
    create_netcdf_point(pool_sicpm, output_path)
    print ('Packing 2D fields of NOAA is complete!!!')
    print ('The output is in sleep, safe and sound!!!')

Start retrieving datasets 1979 (y) - 01 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 02 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 03 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 04 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 05 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 06 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 07 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 08 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 09 (m)
Retrieving datasets successfully and return the variable key!
Start retrieving datasets 1979 (y) - 10 (m)
Re